In [ ]:
import calc_funcs_v1h as cf
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cmocean
import logging

In [ ]:
def add_pcolormesh(da, ax, extents):
    if da.attrs["full_name"].split(" ")[0] == "Difference":
        cmap = cmocean.cm.balance
        vmin = None
        vmax = None
    else:
        cmap = "viridis"
        vmin = (da
                .sel(longitude=slice(extents[0], extents[1]), 
                     latitude=slice(extents[3], extents[2]))
                .min()
               )
        vmax = (da
                .sel(longitude=slice(extents[0], extents[1]), 
                     latitude=slice(extents[3], extents[2]))
                .max()
               )
    ax.set_extent(extents=extents, crs=ccrs.PlateCarree())
    da.plot.pcolormesh(ax = ax, cmap = cmap, transform = ccrs.PlateCarree(),
                       vmin = vmin, vmax = vmax, 
                       cbar_kwargs = {"label": "{} [{}]"
                                      .format(da.attrs["abbreviation"], 
                                              da.attrs["units"])
                                     }
                      )
    ax.set_title(da.attrs["full_name"])
    ax.add_feature(cfeature.COASTLINE)
    ax.gridlines(draw_labels=True, x_inline=False, y_inline=False)

In [ ]:
def add_pcolormesh_cyclic(da, ax, extents):
    if da.attrs["full_name"].split(" ")[0] == "Difference":
        cmap = "twilight_shifted"
        levels = np.arange(-12, 13)
    else:
        cmap = cmocean.cm.phase
        levels = np.arange(0, 25)
    ax.set_extent(extents=extents, crs=ccrs.PlateCarree())
    da.plot.pcolormesh(ax = ax, cmap = cmap, transform = ccrs.PlateCarree(),
                       levels = levels,
                       cbar_kwargs = {"label": "{} [{}]"
                                      .format(da.attrs["abbreviation"], 
                                              da.attrs["units"])
                                     }
                      )
    ax.set_title(da.attrs["full_name"])
    ax.add_feature(cfeature.COASTLINE)
    ax.gridlines(draw_labels=True, x_inline=False, y_inline=False)

In [ ]:
def add_quiver(da_u, da_v, ax, extents):
    var_or_dvar = (da_u
                   .attrs["abbreviation"]
                   .split("(")[-1]
                   .split(")")[0]
                   .split("^")[0]
                   .replace("U", "wv")
                  )
    attrs = cf.da_attrs[var_or_dvar] # This won't work for dvars
    da_mag = xr.DataArray(cf.get_magnitude(da_u, da_v), name = "mag")
    da_u_unit = xr.DataArray(da_u / da_mag, name = "u_unit")
    da_v_unit = xr.DataArray(da_v / da_mag, name = "v_unit")
    ds = xr.merge([da_mag, da_u_unit, da_v_unit])
    ds.plot.quiver(x = "longitude", y = "latitude", ax = ax, 
                   u = "u_unit", v = "v_unit", hue = "mag", 
                   cmap = cmocean.cm.speed, transform = ccrs.PlateCarree(),
                   cbar_kwargs={"label": "{} [{}]"
                                .format(attrs["abbreviation"], 
                                        attrs["wv100"]["units"])
                               }
                  )
    ax.set_title(cf.da_attrs["mean"]["full_name"].format(cf.da_attrs["wv100"]["full_name"]))
    ax.add_feature(cfeature.COASTLINE)
    ax.gridlines(draw_labels=True, x_inline=False, y_inline=False)

In [ ]:
'$mean$$^{diff}$(dU10)'.split("(")[-1].split(")")[0].split("^")[0]

In [ ]:
'dU10^{diff}'.split("(")[-1].split(")")[0].split("^")[0]

In [ ]:
'dU10'.split("(")[-1].split(")")[0].split("^")[0]

In [ ]:
plt.rcParams['text.usetex'] = True

da_mslp_mean = xr.open_dataset(f"../data_processed/era5_mdp_clim_stats_given_var_or_dvar/cfv1h_calc_wa_Jan-1992_Dec-1996_all_era5-mdp_mslp_stats.nc")["mean"]

fig, ax = plt.subplots(1, 1, figsize=(10, 5), 
                       subplot_kw = {"projection": ccrs.PlateCarree()}
                      )

add_pcolormesh(da_mslp_mean, ax, cf.regions["wa"]["extent"])

fig.tight_layout()
plt.show()

In [ ]:
plt.rcParams['text.usetex'] = True

da_mslp_mean = xr.open_dataset(f"../data_processed/era5_mdp_clim_stats_given_var_or_dvar/cfv1h_diff_wa_Jan-1992_Dec-1996_Jan-2002_Dec-2006_all_era5-mdp_mslp_stats.nc")["mean"]

fig, ax = plt.subplots(1, 1, figsize=(10, 5), 
                       subplot_kw = {"projection": ccrs.PlateCarree()}
                      )

add_pcolormesh(da_mslp_mean, ax, cf.regions["wa"]["extent"])

fig.tight_layout()
plt.show()

In [ ]:
plt.rcParams['text.usetex'] = True

da_mslp_hour_max = xr.open_dataset(f"../data_processed/era5_mdp_clim_stats_given_var_or_dvar/cfv1h_calc_wa_Jan-1992_Dec-1996_all_era5-mdp_mslp_stats.nc")["hour_max"]

fig, ax = plt.subplots(1, 1, figsize=(10, 5), 
                       subplot_kw = {"projection": ccrs.PlateCarree()}
                      )

add_pcolormesh_cyclic(da_mslp_hour_max, ax, cf.regions["wa"]["extent"])

fig.tight_layout()
plt.show()

In [ ]:
plt.rcParams['text.usetex'] = True

da_mslp_hour_max = xr.open_dataset(f"../data_processed/era5_mdp_clim_stats_given_var_or_dvar/cfv1h_diff_wa_Jan-1992_Dec-1996_Jan-2002_Dec-2006_all_era5-mdp_mslp_stats.nc")["hour_max"]

fig, ax = plt.subplots(1, 1, figsize=(10, 5), 
                       subplot_kw = {"projection": ccrs.PlateCarree()}
                      )

add_pcolormesh_cyclic(da_mslp_hour_max, ax, cf.regions["wa"]["extent"])

fig.tight_layout()
plt.show()

In [ ]:
# Sample pcolormesh using mean MSLP

plt.rcParams['text.usetex'] = True

da = xr.open_dataset(f"../data_processed/era5_mdp_clim_stats_given_var_or_dvar/cfv1h_calc_wa_Jan-1992_Dec-1996_all_era5-mdp_mslp_stats.nc")["mean"]

extents = cf.regions["wa"]["extent"]
max_da = (da
          .sel(longitude=slice(extents[0], extents[1]), 
               latitude=slice(extents[3], extents[2]))
          .max()
         )
min_da = (da
          .sel(longitude=slice(extents[0], extents[1]), 
               latitude=slice(extents[3], extents[2]))
          .min()
         )

# fig, axes = plt.subplots(4, 2, figsize=(20, 20), 
#                          subplot_kw = {"projection": ccrs.PlateCarree()}
#                         )
# ax = axes[0][0]
fig, ax = plt.subplots(1, 1, figsize=(10, 5), 
                       subplot_kw = {"projection": ccrs.PlateCarree()}
                      )

ax.set_extent(extents=extents, crs=ccrs.PlateCarree())
da.plot.pcolormesh(ax = ax, cmap = "viridis", transform = ccrs.PlateCarree(),
                   vmin = min_da, vmax = max_da, 
                   cbar_kwargs={"label": "{} [{}]"
                                .format(da.attrs["abbreviation"], da.attrs["units"])}
                  )
ax.set_title(da.attrs["full_name"])
ax.add_feature(cfeature.COASTLINE)
ax.gridlines(draw_labels=True, x_inline=False, y_inline=False)

fig.tight_layout()
plt.show()

In [ ]:
# Sample pcolormesh using difference in mean MSLP

plt.rcParams['text.usetex'] = True

da = xr.open_dataset(f"../data_processed/era5_mdp_clim_stats_given_var_or_dvar/cfv1h_diff_wa_Jan-1992_Dec-1996_Jan-2002_Dec-2006_all_era5-mdp_mslp_stats.nc")["mean"]

extents = cf.regions["wa"]["extent"]

# fig, axes = plt.subplots(4, 2, figsize=(20, 20), 
#                          subplot_kw = {"projection": ccrs.PlateCarree()}
#                         )
# ax = axes[0][0]
fig, ax = plt.subplots(1, 1, figsize=(10, 5), 
                       subplot_kw = {"projection": ccrs.PlateCarree()}
                      )

ax.set_extent(extents=extents, crs=ccrs.PlateCarree())
da.plot.pcolormesh(ax = ax, cmap = cmocean.cm.balance, transform = ccrs.PlateCarree(),
                   cbar_kwargs={"label": "{} [{}]"
                                .format(da.attrs["abbreviation"], da.attrs["units"])}
                  )
ax.set_title(da.attrs["full_name"])
ax.add_feature(cfeature.COASTLINE)
ax.gridlines(draw_labels=True, x_inline=False, y_inline=False)

fig.tight_layout()
plt.show()

In [ ]:
# Sample pcolormesh_cyclic using hour_max of MSLP

plt.rcParams['text.usetex'] = True

da = xr.open_dataset(f"../data_processed/era5_mdp_clim_stats_given_var_or_dvar/cfv1h_calc_wa_Jan-1992_Dec-1996_all_era5-mdp_mslp_stats.nc")["hour_max"]

extents = cf.regions["wa"]["extent"]

# fig, axes = plt.subplots(4, 2, figsize=(20, 20), 
#                          subplot_kw = {"projection": ccrs.PlateCarree()}
#                         )
# ax = axes[0][0]
fig, ax = plt.subplots(1, 1, figsize=(10, 5), 
                       subplot_kw = {"projection": ccrs.PlateCarree()}
                      )

ax.set_extent(extents=extents, crs=ccrs.PlateCarree())
da.plot.pcolormesh(ax = ax, cmap = cmocean.cm.phase, transform = ccrs.PlateCarree(),
                   vmin = 0, vmax = 23, levels = np.arange(0, 25), 
                   cbar_kwargs={"label": "{} [{}]"
                                .format(da.attrs["abbreviation"], da.attrs["units"])}
                  )
ax.set_title(da.attrs["full_name"])
ax.add_feature(cfeature.COASTLINE)
ax.gridlines(draw_labels=True, x_inline=False, y_inline=False)

fig.tight_layout()
plt.show()

In [ ]:
# Sample pcolormesh_cyclic using difference in hour_max of MSLP

plt.rcParams['text.usetex'] = True

da = xr.open_dataset(f"../data_processed/era5_mdp_clim_stats_given_var_or_dvar/cfv1h_diff_wa_Jan-1992_Dec-1996_Jan-2002_Dec-2006_all_era5-mdp_mslp_stats.nc")["hour_max"]

extents = cf.regions["wa"]["extent"]

# fig, axes = plt.subplots(4, 2, figsize=(20, 20), 
#                          subplot_kw = {"projection": ccrs.PlateCarree()}
#                         )
# ax = axes[0][0]
fig, ax = plt.subplots(1, 1, figsize=(10, 5), 
                       subplot_kw = {"projection": ccrs.PlateCarree()}
                      )

ax.set_extent(extents=extents, crs=ccrs.PlateCarree())
da.plot.pcolormesh(ax = ax, cmap = "twilight_shifted", transform = ccrs.PlateCarree(),
                   vmin = -12, vmax = 11, levels = np.arange(-12, 13), 
                   cbar_kwargs={"label": "{} [{}]"
                                .format(da.attrs["abbreviation"], da.attrs["units"])}
                  )
ax.set_title(da.attrs["full_name"])
ax.add_feature(cfeature.COASTLINE)
ax.gridlines(draw_labels=True, x_inline=False, y_inline=False)

fig.tight_layout()
plt.show()

In [ ]:
# Sample quiver using mean WV100

plt.rcParams['text.usetex'] = True

ds = xr.open_dataset(f"../data_processed/era5_mdp_clim_stats_given_var_or_dvar/cfv1h_calc_wa_Jan-1992_Dec-1996_all_era5-mdp_wv100_stats.nc")
ds["mean_mag"] = cf.get_magnitude(ds["mean_u"], ds["mean_v"])
ds["mean_u_unit"] = ds["mean_u"] / ds["mean_mag"]
ds["mean_v_unit"] = ds["mean_v"] / ds["mean_mag"]

extents = cf.regions["wa"]["extent"]

# fig, axes = plt.subplots(4, 2, figsize=(20, 20), 
#                          subplot_kw = {"projection": ccrs.PlateCarree()}
#                         )
# ax = axes[0][0]
fig, ax = plt.subplots(1, 1, figsize=(10, 5), 
                       subplot_kw = {"projection": ccrs.PlateCarree()}
                      )

ax.set_extent(extents=extents, crs=ccrs.PlateCarree())
ds.plot.quiver(x = "longitude", y = "latitude", ax = ax, 
               u = "mean_u_unit", v = "mean_v_unit", hue = "mean_mag", 
               cmap = cmocean.cm.speed, transform = ccrs.PlateCarree(),
               cbar_kwargs={"label": "{} [{}]"
                            .format(cf.da_attrs["wv100"]["abbreviation"], 
                                    cf.da_attrs["wv100"]["units"])
                           },
               # add_guide = False
              )
ax.set_title(cf.da_attrs["mean"]["full_name"].format(cf.da_attrs["wv100"]["full_name"]))
ax.add_feature(cfeature.COASTLINE)
ax.gridlines(draw_labels=True, x_inline=False, y_inline=False)

fig.tight_layout()
plt.show()

In [ ]:
# Sample quiver using difference in mean WV100

plt.rcParams['text.usetex'] = True

ds = xr.open_dataset(f"../data_processed/era5_mdp_clim_stats_given_var_or_dvar/cfv1h_diff_wa_Jan-1992_Dec-1996_Jan-2002_Dec-2006_all_era5-mdp_wv100_stats.nc")
ds["mean_mag"] = cf.get_magnitude(ds["mean_u"], ds["mean_v"])
ds["mean_u_unit"] = ds["mean_u"] / ds["mean_mag"]
ds["mean_v_unit"] = ds["mean_v"] / ds["mean_mag"]

extents = cf.regions["wa"]["extent"]

# fig, axes = plt.subplots(4, 2, figsize=(20, 20), 
#                          subplot_kw = {"projection": ccrs.PlateCarree()}
#                         )
# ax = axes[0][0]
fig, ax = plt.subplots(1, 1, figsize=(10, 5), 
                       subplot_kw = {"projection": ccrs.PlateCarree()}
                      )

ax.set_extent(extents=extents, crs=ccrs.PlateCarree())
ds.plot.quiver(x = "longitude", y = "latitude", ax = ax, 
               u = "mean_u_unit", v = "mean_v_unit", hue = "mean_mag", 
               cmap = cmocean.cm.tempo, transform = ccrs.PlateCarree(),
               cbar_kwargs={"label": "{} [{}]"
                            .format(cf.da_attrs["wv100"]["abbreviation"], 
                                    cf.da_attrs["wv100"]["units"])
                           },
               # add_guide = False
              )
ax.set_title("Difference in " + cf.da_attrs["mean"]["full_name"].format(cf.da_attrs["wv100"]["full_name"]))
ax.add_feature(cfeature.COASTLINE)
ax.gridlines(draw_labels=True, x_inline=False, y_inline=False)

fig.tight_layout()
plt.show()

In [ ]:
# from dask.distributed import Client
# client = Client()
# client

In [ ]:
# client.close()

In [ ]:
import ipywidgets as ipw
import hvplot.xarray # noqa
import hvplot.pandas # noqa
import panel as pn
import pandas as pd
import panel.widgets as pnw
import xarray as xr

In [ ]:
%%time
cf.calc_glass_mean_clim("wa", "Jun-2000", "Aug-2005", [8, 6,7])

In [ ]:
%%time
cf.calc_era5_mdp_clim_given_var_or_dvar("sa", "Dec-1994", "Feb-2000", "djf", "wv10")

In [ ]:
%%time
cf.calc_era5_mdp_clim_stats_given_var_or_dvar("sa", "Dec-1994", "Feb-2000", "jja", "dwv100")

In [ ]:
%%time
cf.calc_era5_wsd_clim("wa", "Jun-2000", "Aug-2005", [1,3,5,7,9,11])

In [ ]:
%%time
cf.calc_diff(cf.calc_era5_mdp_clim_given_var_or_dvar, "ca", "Jan-1985", "Dec-1994", "Jan-1995", "Jan-2004", "all", "dnac")

In [ ]:
test1 = xr.open_dataset(f"../data_processed/glass_mean_clim/{cf.calc_funcs_ver}_calc_wa_Jun-2000_Aug-2005_6-7-8_glass-mean_avhrr.nc")
test1

In [ ]:
test1["mlai"].plot()

In [ ]:
test1["mfapar"].plot()

In [ ]:
test2 = xr.open_dataset(f"../data_processed/era5_mdp_clim_given_var_or_dvar/{cf.calc_funcs_ver}_calc_sa_Dec-1994_Feb-2000_djf_era5-mdp_wv10.nc")
test2

In [ ]:
test2["u10"].sel(hour=21).plot()

In [ ]:
test2["v10"].sel(hour=21).plot()

In [ ]:
test3 = xr.open_dataset(f"../data_processed/era5_mdp_clim_given_var_or_dvar/{cf.calc_funcs_ver}_diff_ca_Jan-1985_Dec-1994_Jan-1995_Jan-2004_all_era5-mdp_dnac.nc")
test3

In [ ]:
test3["dnse"].sel(hour=21).plot()

In [ ]:
test3["dvidmf"].sel(hour=21).plot()

In [ ]:
test3["dvidcfwf"].sel(hour=21).plot()

In [ ]:
test3["dvidclwf"].sel(hour=21).plot()

In [ ]:
test3["dtcwv"].sel(hour=21).plot()

In [ ]:
test3["dnac"].sel(hour=21).plot()

In [ ]:
test4 = xr.open_dataset(f"../data_processed/era5_mdp_clim_stats_given_var_or_dvar/{cf.calc_funcs_ver}_calc_sa_Dec-1994_Feb-2000_jja_era5-mdp_dwv100_stats.nc")
test4

In [ ]:
test4["hour_max"].plot()

In [ ]:
test4["hour_min"].plot()

In [ ]:
test4["max_u"].plot()

In [ ]:
test4["max_v"].plot()

In [ ]:
test4["min_u"].plot()

In [ ]:
test4["min_v"].plot()

In [ ]:
test4["mean_u"].plot()

In [ ]:
test4["mean_v"].plot()

In [ ]:
test4["range"].plot()

In [ ]:
# test4["max"].plot()

In [ ]:
# test4["min"].plot()

In [ ]:
# test4["mean"].plot()

In [ ]:
test5 = xr.open_dataset(f"../data_processed/era5_wsd_clim/{cf.calc_funcs_ver}_calc_wa_Jun-2000_Aug-2005_1-3-5-7-9-11_era5-wsd.nc")
test5

In [ ]:
test5["ws100_mean"].plot()

In [ ]:
test5["ws100_std"].plot()

In [ ]:
test5["c100"].plot()

In [ ]:
test5["k100"].plot()

In [ ]:
test5["eroe100"].plot()

In [ ]:
test5["tgcf100"].plot()

In [ ]:
test5["eroe100"].where(test5["eroe100"]==test5["eroe100"].max(), drop = True).squeeze()

In [ ]:
%%time
cf.calc_diff(cf.calc_glass_mean_clim, "wa", "Jan-1985", "Dec-1990", "Jan-2005", "Dec-2010", "all")

In [ ]:
%%time
cf.calc_diff(cf.calc_era5_mdp_clim_given_var_or_dvar, "wa", "Jan-1985", "Dec-1990", "Jan-2005", "Dec-2010", "all", "nac")

In [ ]:
%%time
cf.calc_diff(cf.calc_era5_mdp_clim_stats_given_var_or_dvar, "wa", "Jan-1985", "Dec-1990", "Jan-2005", "Dec-2010", "all", "wv10")

In [ ]:
%%time
cf.calc_diff(cf.calc_era5_wsd_clim, "wa", "Jan-1985", "Dec-1990", "Jan-2005", "Dec-2010", "all")

In [ ]:
test6 = xr.open_dataset(f"../data_processed/glass_mean_clim/{cf.calc_funcs_ver}_diff_wa_Jan-1985_Dec-1990_Jan-2005_Dec-2010_all_glass-mean_avhrr.nc")
test6

In [ ]:
test6["mlai"].plot()

In [ ]:
test6["mfapar"].plot()

In [ ]:
test7 = xr.open_dataset(f"../data_processed/era5_mdp_clim_given_var_or_dvar/{cf.calc_funcs_ver}_diff_wa_Jan-1985_Dec-1990_Jan-2005_Dec-2010_all_era5-mdp_nac.nc")
test7

In [ ]:
test7["nse"].sel(hour=21).plot()

In [ ]:
test7["vidmf"].sel(hour=21).plot()

In [ ]:
test7["vidcfwf"].sel(hour=21).plot()

In [ ]:
test7["vidclwf"].sel(hour=21).plot()

In [ ]:
test7["tcwv"].sel(hour=21).plot()

In [ ]:
test7["nac"].sel(hour=21).plot()

In [ ]:
test8 = xr.open_dataset(f"../data_processed/era5_mdp_clim_stats_given_var_or_dvar/{cf.calc_funcs_ver}_diff_wa_Jan-1985_Dec-1990_Jan-2005_Dec-2010_all_era5-mdp_wv10_stats.nc")
test8

In [ ]:
test8["hour_max"].plot()

In [ ]:
test8["hour_min"].plot()

In [ ]:
test8["max_u"].plot()

In [ ]:
test8["max_v"].plot()

In [ ]:
test8["min_u"].plot()

In [ ]:
test8["min_v"].plot()

In [ ]:
test8["mean_u"].plot()

In [ ]:
test8["mean_v"].plot()

In [ ]:
test8["range"].plot()

In [ ]:
test9 = xr.open_dataset(f"../data_processed/era5_wsd_clim/{cf.calc_funcs_ver}_diff_wa_Jan-1985_Dec-1990_Jan-2005_Dec-2010_all_era5-wsd.nc")
test9

In [ ]:
test9["ws100_mean"].plot()

In [ ]:
test9["ws100_std"].plot()

In [ ]:
test9["c100"].plot()

In [ ]:
test9["k100"].plot()

In [ ]:
test9["eroe100"].plot()

In [ ]:
test9["tgcf100"].plot()

In [ ]:
test9["eroe100"].where(test9["eroe100"]==test9["eroe100"].max(), drop = True).squeeze()

In [ ]:
%%time
cf.calc_era5_land_elev()

In [ ]:
test10 = xr.open_dataset(f"../data_processed/era5_land_elev/{cf.calc_funcs_ver}_calc_global_static_land-elev.nc")
test10

In [ ]:
test10["lse"].plot()

In [ ]:
%%time
cf.calc_glass_rolling_avg_of_annual_diff("wa", 1984, 1987, 7)

In [ ]:
test11 = xr.open_dataset(f"../data_processed/glass_rolling_avg_of_annual_diff/{cf.calc_funcs_ver}_calc_wa_1984_1987_7-year_glass-rolling-diff_pref-avhrr.nc")
test11

In [ ]:
test11["mlai"].isel(year=0).plot()

In [ ]:
test11["mfapar"].isel(year=0).plot()

In [ ]:
%%time
cf.calc_glass_rolling_avg_of_annual_diff("wa", 1983, 2019, 5)

In [ ]:
%%time
cf.calc_glass_rolling_avg_of_annual_diff("wa", 1984, 2018, 7)

In [ ]:
test12 = xr.open_dataset(f"../data_processed/glass_rolling_avg_of_annual_diff/{cf.calc_funcs_ver}_calc_wa_1983_2019_5-year_glass-rolling-diff_pref-avhrr.nc")
test12

In [ ]:
test12["mlai"].interactive.sel(year=pnw.DiscreteSlider).plot(cmap = "RdBu", vmin = -1, vmax = 1)

In [ ]:
test13 = xr.open_dataset(f"../data_processed/glass_rolling_avg_of_annual_diff/{cf.calc_funcs_ver}_calc_wa_1984_2018_7-year_glass-rolling-diff_pref-avhrr.nc")
test13

In [ ]:
test13["mlai"].interactive.sel(year=pnw.DiscreteSlider).plot(cmap = "RdBu", vmin = -1, vmax = 1)

In [ ]:
%%time
cf.create_all_possible_calc_data_files("wa", "Jan-1992", "Dec-1996", "all")

In [ ]:
%%time
cf.create_all_possible_calc_data_files("wa", "Jan-2002", "Dec-2006", "all")

In [ ]:
%%time
cf.create_all_possible_diff_data_files("wa", "Jan-1992", "Dec-1996", "Jan-2002", "Dec-2006", "all")

In [ ]:
test14 = xr.open_dataset(f"../data_processed/glass_mean_clim/{cf.calc_funcs_ver}_diff_wa_Jan-1992_Dec-1996_Jan-2002_Dec-2006_all_glass-mean_avhrr.nc")
test14["mlai"].plot()

In [ ]:
test15 = xr.open_dataset(f"../data_processed/era5_wsd_clim/{cf.calc_funcs_ver}_diff_wa_Jan-1992_Dec-1996_Jan-2002_Dec-2006_all_era5-wsd.nc")
test15["ws10_mean"].plot()